In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
true_to_seires = pd.read_csv('../output/inv_y_rnn.csv', index_col=0)
print(true_to_seires.shape)

true_to_seires = true_to_seires.iloc[-17521:, :]
print(true_to_seires.shape)

(19249, 96)
(17521, 96)


In [4]:
# sum(true_to_seires.values - true_to_seires2.values)

In [5]:
lr_raw = pd.read_csv('../output/inv_yhat_lr.csv', index_col=0)
mlp_raw = pd.read_csv('../output/inv_yhat_mlp.csv', index_col=0)
rnn_raw = pd.read_csv('../output/inv_yhat_rnn.csv', index_col=0)
lstm_raw = pd.read_csv('../output/inv_yhat_lstm.csv', index_col=0)
conv_mlp_raw = pd.read_csv('../output/inv_yhat_cnn.csv', index_col=0)
conv_rnn_raw = pd.read_csv('../output/inv_yhat_rcnn.csv', index_col=0)

In [6]:
print(lr_raw.shape, mlp_raw.shape, rnn_raw.shape, lstm_raw.shape, conv_mlp_raw.shape, conv_rnn_raw.shape)

(19249, 96) (19249, 96) (19249, 96) (19249, 96) (19249, 96) (19249, 96)


In [7]:
lr = lr_raw.iloc[-17521:, :]
mlp = mlp_raw.iloc[-17521:, :]
rnn = rnn_raw.iloc[-17521:, :]
lstm = lstm_raw.iloc[-17521:, :]
conv_mlp = conv_mlp_raw.iloc[-17521:, :]
conv_rnn = conv_rnn_raw.iloc[-17521:, :]

In [8]:
print(lr.shape)
print(mlp.shape)
print(rnn.shape)
print(lstm.shape)
print(conv_mlp.shape)
print(conv_rnn.shape)

(17521, 96)
(17521, 96)
(17521, 96)
(17521, 96)
(17521, 96)
(17521, 96)


### MAE vs location (24 hour aheae)

In [25]:
def mae(y_true, predictions):
    y_true, predictions = np.array(y_true), np.array(predictions)
    return np.mean(np.abs(y_true - predictions))

In [38]:
cols = {'S1':-4, 'S25A':-3, 'S25B':-2, 'S26':-1}

for loc, col in cols.items():
    print('MAE (24 hr ahead) at', loc, ':')
    print(f'{mae(true_to_seires.iloc[:, col], lr.iloc[:, col]):.3f}')
    print(f'{mae(true_to_seires.iloc[:, col], mlp.iloc[:, col]):.3f}')
    print(f'{mae(true_to_seires.iloc[:, col], rnn.iloc[:, col]):.3f}')
    print(f'{mae(true_to_seires.iloc[:, col], lstm.iloc[:, col]):.3f}')
    print(f'{mae(true_to_seires.iloc[:, col], conv_mlp.iloc[:, col]):.3f}')
    print(f'{mae(true_to_seires.iloc[:, col], conv_rnn.iloc[:, col]):.3f}')

MAE (24 hr ahead) at S1 :
0.240
0.160
0.135
0.146
0.107
0.072
MAE (24 hr ahead) at S25A :
0.362
0.106
0.139
0.136
0.093
0.085
MAE (24 hr ahead) at S25B :
0.122
0.105
0.150
0.160
0.090
0.082
MAE (24 hr ahead) at S26 :
0.461
0.258
0.169
0.167
0.207
0.184


### HEC-RAS vs location

In [9]:
error_ras = pd.read_csv('../output/ras_1920.csv', index_col=0)
error_ras.reset_index(drop=True, inplace=True)
error_ras

,S1_OBS,S1_RAS,S1_ERROR,S25A_TW_OBS,S25A_TW_RAS,S25A_TW_ERROR,S25B_TW_OBS,S25B_TW_RAS,S25B_TW_ERROR,S26_TW_OBS,S26_TW_RAS,S26_TW_ERROR
0,-0.32,-0.54,-0.22,-0.31,-0.54,-0.23,-0.28,-0.53,-0.25,-0.30,-0.54,-0.24
1,0.17,-0.05,-0.22,0.14,-0.05,-0.19,0.19,-0.05,-0.24,0.19,-0.05,-0.24
2,0.80,0.62,-0.18,0.76,0.61,-0.15,0.83,0.63,-0.20,0.80,0.63,-0.17
3,1.26,1.09,-0.17,1.27,1.10,-0.17,1.31,1.10,-0.21,1.28,1.08,-0.20
4,1.68,1.51,-0.17,1.67,1.53,-0.14,1.70,1.51,-0.19,1.69,1.52,-0.17
...,...,...,...,...,...,...,...,...,...,...,...,...
17539,0.18,0.21,0.03,0.13,0.22,0.09,0.26,0.22,-0.04,0.17,0.20,0.03
17540,0.52,0.69,0.17,0.52,0.68,0.16,0.60,0.67,0.07,0.54,0.79,0.25
17541,1.14,1.13,-0.01,1.18,1.14,-0.04,1.17,1.13,-0.04,1.25,1.14,-0.11
17542,1.62,1.67,0.05,1.65,1.67,0.02,1.70,1.69,-0.01,1.67,1.67,0.00


In [11]:
error_ras_concat = pd.concat([error_ras.loc[:, 'S1_ERROR'], error_ras.loc[:, 'S25A_TW_ERROR'], error_ras.loc[:, 'S25B_TW_ERROR'], error_ras.loc[:, 'S26_TW_ERROR']], axis=1)
error_ras_concat = pd.DataFrame(error_ras_concat)
# error_ras_concat

In [12]:
ws_ras_concat = pd.concat([error_ras.loc[:, 'S1_RAS'], error_ras.loc[:, 'S25A_TW_RAS'], error_ras.loc[:, 'S25B_TW_RAS'], error_ras.loc[:, 'S26_TW_RAS']], axis=1)
ws_ras_concat = pd.DataFrame(ws_ras_concat)
ws_ras_concat

,S1_RAS,S25A_TW_RAS,S25B_TW_RAS,S26_TW_RAS
0,-0.54,-0.54,-0.53,-0.54
1,-0.05,-0.05,-0.05,-0.05
2,0.62,0.61,0.63,0.63
3,1.09,1.10,1.10,1.08
4,1.51,1.53,1.51,1.52
...,...,...,...,...
17539,0.21,0.22,0.22,0.20
17540,0.69,0.68,0.67,0.79
17541,1.13,1.14,1.13,1.14
17542,1.67,1.67,1.69,1.67


In [13]:
ws_true_concat = pd.concat([error_ras.loc[:, 'S1_OBS'], error_ras.loc[:, 'S25A_TW_OBS'], error_ras.loc[:, 'S25B_TW_OBS'], error_ras.loc[:, 'S26_TW_OBS']], axis=1)
ws_true_concat = pd.DataFrame(ws_true_concat)
ws_true_concat

,S1_OBS,S25A_TW_OBS,S25B_TW_OBS,S26_TW_OBS
0,-0.32,-0.31,-0.28,-0.30
1,0.17,0.14,0.19,0.19
2,0.80,0.76,0.83,0.80
3,1.26,1.27,1.31,1.28
4,1.68,1.67,1.70,1.69
...,...,...,...,...
17539,0.18,0.13,0.26,0.17
17540,0.52,0.52,0.60,0.54
17541,1.14,1.18,1.17,1.25
17542,1.62,1.65,1.70,1.67


In [24]:
stations = ['S1', 'S25A', 'S25B', 'S26']
for i in range(len(stations)):
    print("Station:", stations[i])
    each_station = error_ras_concat.iloc[:, i]
    print(f'{len(each_station[each_station > 0.5]) / len(each_station) * 100 + len(each_station[each_station < -0.5]) / len(each_station) * 100:.3f}' + '%')
    print('     ')

Station: S1
2.599%
     
Station: S25A
3.249%
     
Station: S25B
4.417%
     
Station: S26
3.585%
     
